<a href="https://colab.research.google.com/github/SiluniKeerthiratne/DSGP/blob/nisal/%20v3%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Mount drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import dependancies
import tensorflow as tf
import os
import cv2
from tqdm import tqdm
from tqdm import tqdm_notebook

In [ ]:
# Assign directory
root_dir = r'C:\Users\nisal\DSGP\all\FruitDataset'

fruit_classes = ['banana','mango','tomato']
ripeness_classes = ['0 unripe', '1 partially ripe', '2 ripe']

X = []
y = []

for fruit_class in fruit_classes:
    for ripeness_class_label, ripeness_class in enumerate(ripeness_classes):
        class_path = os.path.join(root_dir, fruit_class, ripeness_class)
        for img_name in tqdm(os.listdir(class_path)):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (244, 244))
            X.append(img)
            y.append(ripeness_class_label)

In [ ]:
# Splitting data into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)
print("X_test Shape: ", X_test.shape)
print("y_test Shape: ", y_test.shape)

In [ ]:
# Build model
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras import regularizers

img_rows, img_cols = 244, 244

In [ ]:
# Load 'InceptionV3' model
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))

In [ ]:
# Freeze the layers
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# Custom head
def custom_head(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(top_model)
    top_model = Dropout(0.5)(top_model)  # Add dropout for regularization
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model

In [ ]:
# Add custom head on top of InceptionV3
num_classes = len(ripeness_classes)
FC_Head = custom_head(inception, num_classes)
model = Model(inputs=inception.input, outputs=FC_Head)

In [ ]:
# Model summary
print(model.summary())

In [ ]:
# Model compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train,epochs=25,verbose=1,initial_epoch=0,validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Get true labels
true_labels = y_test

# Get predictions
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=-1)

# Calculate accuracy
accuracy = np.mean(true_labels == predicted_labels)*100
print("Accuracy:", accuracy,"%")

In [ ]:
# Calculate precision, recall, f1-score, and support
ripeness_classes_names = [str(i) for i in range(len(ripeness_classes))]  # Assuming class indices as numbers
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=ripeness_classes_names))

In [ ]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
# Visualization of Training History
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

In [ ]:
# Save model
model.save(r'C:\Users\nisal\DSGP\all\v3trained_model.h5')